In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

file_dir = ''
labels = pd.read_csv(file_dir + 'labels-1.csv')
display(labels)



In [ ]:
DS = 8             # Downsample rate, must be a multiple of 30976

if 30976/DS % 1 > 0:
    print("Downsample rate is not a multiple of 30976")
    DS = 1
    im_size = 30976
else:
    im_size = int(30976/DS)


data = np.zeros([609, im_size])


for i, file_name in enumerate(labels['Filename']):
        img = np.mean(matplotlib.image.imread(file_dir + file_name),axis=2).reshape(-1)
        data[i,:] = img[::DS]            # Downsample the image

In [ ]:
Train_size=450
train=labels.sample(n=Train_size,replace=False,random_state=255)
test=labels.drop(train.index)
Y_train=train['nWBV']
X_train=data[train.index]
Y_test=test['nWBV']
X_test=data[test.index]

In [ ]:
def RSS(y,Y):
    y = np.matrix(y).reshape(-1,1)
    Y = np.matrix(Y).reshape(-1,1)
    
    return (y-Y).T*(y-Y)

def RMS(y,Y):    
    return np.sqrt(RSS(y,Y))/len(y)

def Rs(y,Y): 
    y = np.matrix(y).reshape(-1,1)
    Y = np.matrix(Y).reshape(-1,1)
    
    return 1 - RSS(y,Y)/((Y - np.mean(Y)).T*(Y - Y.mean()))

In [ ]:
import statsmodels.api as sm
r_sqds = []
alphas = np.linspace(0,.1,50)

for alp in alphas:
    ols = sm.OLS(Y_train, sm.add_constant(X_train))
    ols_result = ols.fit_regularized(L1_wt=0,alpha=alp)

    r_sqds.append(Rs(ols_result.predict(sm.add_constant(X_test,has_constant='add')),Y_test).A1)



In [ ]:
r_sqds_m=np.max(r_sqds)
index=np.argmax(r_sqds)
plt.plot(alphas,r_sqds)

In [ ]:
Train_size=450
train=labels.sample(n=Train_size,replace=False,random_state=255)
test=labels.drop(train.index)
Y_train=train['nWBV']
X_train=data[train.index]
Y_test=test['nWBV']
X_test=data[test.index]
r_sqds = []
rss_s = []
X_train1=pd.DataFrame(X_train)

alphas = np.linspace(0,0.01,10)

for alp in alphas:
    ols = sm.OLS(Y_train, sm.add_constant(X_train))
    ols_result = ols.fit_regularized(L1_wt=1,alpha=alp)
    
    r_sqds.append(Rs(ols_result.predict(sm.add_constant(X_test,has_constant='add')),Y_test).A1)
    rss_s.append(RMS(ols_result.predict(sm.add_constant(X_test,has_constant='add')),Y_test).A1)
    
print(np.max(r_sqds))
print(np.max(rss_s))
plt.plot(alphas,r_sqds)